In [1]:
import numpy as np 
from scipy import optimize 
import matplotlib.pyplot as plt
import matplotlib
import PyQt5
import PySpin

/home/nayak03/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
%matplotlib qt

In [3]:
IMAGE_SHAPE = (2840, 2840)
image = np.fromfile('/home/nayak03/DIS_QAQC/birfurcation_test3_exp16_gain0/birfurcation_test3_exp16_gain0_0999.raw', dtype=np.uint16, sep="").reshape(IMAGE_SHAPE)
plt.imshow(image, cmap="RdBu")
plt.pause(2)
plt.close()

In [4]:
def deg3_poly(x, a, b, c, d):
    return a*(x**3) + b*(x ** 2) + c*(x) + d

In [5]:
def calibration_plot(img): 
    x_proj = np.mean(img, axis=0)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    plt.sca(axes[0])
    plt.title("Image")
    plt.imshow(img, cmap='RdBu')
    plt.colorbar()

    plt.sca(axes[1])
    plt.xlabel("pixel coordinate (x-axis)")
    plt.ylabel("mean value over y-axis")
    plt.plot(np.arange(2840), x_proj, '.')
    plt.axhline(y=np.mean(x_proj), color='red', label='mean')
    plt.axhline(y=np.mean(x_proj) + (np.mean(x_proj)*0.005), color='orange', label='mean+-0.5%', linestyle='dashed')
    plt.axhline(y=np.mean(x_proj) - (np.mean(x_proj)*0.005), color='orange', linestyle='dashed')

    popt, pcov = optimize.curve_fit(deg3_poly, np.arange(2840), x_proj)
    x = np.arange(2840)
    a, b, c, d = popt 
    plt.plot(x, deg3_poly(x, *popt), color='purple', label= f"Parameters:\n$a = {a:.2e}$, $b = {b:.2e}$ \n $c = {c:.2e}$, $d = {d:.0f}$")
    plt.title("Calibration Plot")
    
    plt.legend()

In [6]:
calibration_plot(image)
plt.show()
plt.pause(10)
plt.close()

In [7]:
def set_buffer_handling_mode(cam):
    try:
        result = True
        s_nodemap = cam.GetTLStreamNodeMap()
    
        # Set the Buffer Handling Mode to Newest Only for LIVE capture
    
        node_stream_buffer_handling_mode = PySpin.CEnumerationPtr(s_nodemap.GetNode('StreamBufferHandlingMode'))
        if not PySpin.IsReadable(node_stream_buffer_handling_mode) or not PySpin.IsWritable(node_stream_buffer_handling_mode):
            print('Unable to access Stream Buffer Handling (node retrieval). Non-fatal error...')
            return False
        node_stream_buffer_handling_mode_newest_only = node_stream_buffer_handling_mode.GetEntryByName('NewestOnly')
        if not PySpin.IsReadable(node_stream_buffer_handling_mode_newest_only):
            print('Unable to access Stream Buffer Handling (enum entry retrieval). Non-fatal error...')
            return False
        
        stream_buffer_handling_mode_newest_only = node_stream_buffer_handling_mode_newest_only.GetValue()
        
        # Set integer value from entry node as new value of enumeration node
        node_stream_buffer_handling_mode.SetIntValue(stream_buffer_handling_mode_newest_only)
        
        print('Set Stream Buffer Handling Mode to Newest Only...')
    
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False
    
    return result

In [8]:
def main(): 
    global system
    system = PySpin.System.GetInstance()

    global cam_list
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()
    global cam
    if num_cameras > 1:
        print("Warning! More than one camera was found!")

    cam = cam_list.GetByIndex(0)
    cam.Init()
    print(f"The rest of the script will be run with camera of Index 0 and DeviceID {cam.DeviceID.GetValue()}.")

    set_buffer_handling_mode(cam)

    cam.BeginAcquisition()

    i = 0

    while(i < 20): 

        print(f"Acquiring image {i+1}...")
        img = cam.GetNextImage()

        if img.IsIncomplete():
            print(f"Image {i+1} is incomplete! Status: {img.GetImageStatus()}")

        img_array = img.GetNDArray()  
        calibration_plot(img_array)
        plt.show()
        plt.pause(2)
        plt.close()

        img.Release()

        i += 1
    cam.EndAcquisition() 

In [9]:
try: 
    cam.EndAcquisition() 
finally: 
    main()

The rest of the script will be run with camera of Index 0 and DeviceID 23431900.
Set Stream Buffer Handling Mode to Newest Only...
Acquiring image 1...
Acquiring image 2...
Acquiring image 3...
Acquiring image 4...
Acquiring image 5...
Acquiring image 6...
Acquiring image 7...
Acquiring image 8...
Acquiring image 9...
Acquiring image 10...
Acquiring image 11...
Acquiring image 12...
Acquiring image 13...
Acquiring image 14...
Acquiring image 15...
Acquiring image 16...
Acquiring image 17...
Acquiring image 18...
Acquiring image 19...
Acquiring image 20...


NameError: name 'cam' is not defined